## Model's conversion

### Torch2ONNX Demo

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Test3DConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_3d = nn.Sequential(
            nn.Conv3d(32, 32, 3, 1, 1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.Conv3d(32, 32, 3, 1, 1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.Conv3d(32, 1, 3, 1, 1),
        )
        self.conv_2d = nn.Sequential(
            nn.Conv2d(24, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )

    def forward(self, feature_3d):
        x = self.conv_3d(feature_3d)
        x = F.softmax(x.squeeze(1), dim=1)
        x = self.conv_2d(x)
        return x

In [4]:
model = Test3DConv().eval()
data = torch.rand(1, 32, 24, 68, 120)

In [ ]:
torch.onnx.export(
    model,
    (data,),
    "test_case_3dconv.onnx",
    verbose=True,
    input_names=["3d_feature"], # specify inputs
    output_names=["2d_feature"], # specify outputs
    opset_version=9 # specify op set version
    )

### ONNX2TF Demo

Install [onnx2tf](https://github.com/onnx/onnx-tensorflow):

1. Install ONNX master branch from source.
2. Install TensorFlow >= 2.6.0,tensorflow-probability and tensorflow-addons. (Note TensorFlow 1.x is no longer supported)
3. Run git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow.
4. Run pip install -e ..

In [ ]:
import onnx

from onnx_tf.backend import prepare

onnx_model = onnx.load("input_path")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("output_path")  # export the TF protobuf file

### TF2TFlite Demo(Frozen pb)

click [here](https://www.tensorflow.org/lite/convert) for more details.

In [8]:
# prepare the frozen pb file / saved model
# suppose you have installed TF2.x

In [9]:
import tensorflow as tf

In [10]:
if tf.__version__ < '2.0.0':
    print("For tf 1.x, see:\n https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/r1/convert/python_api.md")
else:
    # manually specify the inputs and outputs
    model = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(PB_PATH,inputs,outputs)
    with open('foo.tflite', 'wb') as f:
        f.write(model)

### TF/TFLite2ONNX

[tf2onnx](https://github.com/onnx/tensorflow-onnx) should be installed first.

```bash
pip install -U tf2onnx
```

#### FrozenPB

指定pb文件，inputs/outputs的tensor name。

```bash
python -m tf2onnx.convert --input yolo.pb --inputs input:0 --outputs ouput:0 --output yolo.onnx
```

其他的格式到onnx，见：[here](https://github.com/onnx/tensorflow-onnx#getting-started)